In [1]:
import collections
import random
import gym
import numpy as np

In [2]:
from cartPoleDqn import DQN

In [3]:
class Agent:
    def __init__(self, env: gym.Env):
        # DQN Env Variables
        self.env = env
        self.observations = self.env.observation_space.shape
        self.actions = self.env.action_space.n
        # DQN Agent Variables
        self.replay_buffer_size = 50_000
        self.train_start = 1_000 
            # Ab wie vielen gemachten Spielzügen im Replay-Buffer soll mit dem Training begonnen werden?
            # Das Netzwerk wird so nach 1_000 Spielzügen eingeschaltet
        self.memory = collections.deque(maxlen=self.replay_buffer_size)
            # Eine Liste an der an beiden Seiten etwas ändern kann
            # Ist der Speicher erstmal voll, werden die Daten von rechts nach links verschoben, 
            # bzw. die ältesten Daten werden zugunsten der neuen Daten gelöscht
        self.gamma = 0.95
        self.epsilon = 1.0
            # Wie viel Prozent der der Aktionen sollen schon zu Beginn zufällig gewählt sein?
            # Dieser Wert wird im Laufe des Trainings reduziert bis epsilon_min
        self.epsilon_min = 0.01
            # Minimaler Prozentsatz um zufällige Aktionen auszuführen
            # In einem Prozent der Fälle wollen wir noch eine zufällige Aktion haben
        self.epsilon_decay = 0.999 
            # Je näher an der 1, desto mehr Spielzüge benötigt man,
            # um mit epsilon an epsilon_min anzukommen
        # DQN Network Variables
        self.state_shape = self.observations
        self.learning_rate = 1e-3
        self.dqn = DQN(
            self.state_shape,
            self.actions,
            self.learning_rate
        )
        self.target_dqn = DQN(
            self.state_shape,
            self.actions,
            self.learning_rate
        )
        self.target_dqn.update_model(self.dqn)
        self.batch_size = 32

    def get_action(self, state):
        if np.random.rand() <= self.epsilon: 
            return np.random.randint(self.actions)
        else:
            return np.argmax(self.dqn(state)) # Die Aktion mit dem höchsten q-Value

    def train(self, num_episodes):
        pass

    def remember(self, state, action, reward, next_state, done ):
        pass
    
    def replay(self):
        pass

    def play(self, num_episodes, render=True):
        pass

In [4]:
if __name__ == "__main__":
    env = gym.make("CartPole-v1")
    agent = Agent(env)